<a href="https://colab.research.google.com/github/licTomasPerez/Spin-Chains-/blob/main/HS_projected_evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qutip
!pip install scipy
!pip install matplotlib.inline
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.7 MB 8.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import pickle
import math, cmath

In [ ]:
def prod_basis(b1, b2):
  return [qutip.tensor(b,s) for b in b1 for s in b2]

def scalar_prod(op1, op2, rho0 = None):
  if op1.dims[0][0]!=op2.dims[0][0]:
    return "Incompatible Qobj dimensions"
  if rho0 is None:
    rho0 = qutip.qeye(op1.dims[0])/op1.dims[0][0]
  return ((op1.dag()*op2))

def base_orth(ops, rho0):
  dim = ops[0].dims[0][0]
  basis = []
  for op in ops: 
    alpha = [scalar_prod(op2, op, rho0) for op2 in basis]
    op_mod = op - sum([c*op2 for c, op2, in zip(alpha, basis)])
    op_mod = op_mod/(np.sqrt(scalar_prod(op_mod,op_mod,rho0)))
    basis.append(op_mod)
  return basis

def logM(rho):
  eigvals, eigvecs = rho.eigenstates()
  return sum([np.log(vl)*vc*vc.dag() for vl, vc in zip(eigvals, eigvecs) if vl > 0])

def sqrtM(rho):
  eigvals, eigvecs = rho.eigenstates()
  return sum([(abs(vl)**.5)*vc*vc.dag() for vl, vc in zip(eigvals, eigvecs)])

def proj_op(K, basis, rho0):
  return sum([scalar_prod(b, K,rho0) * b for b in basis])

def rel_entropy(rho, sigma):
    val = (rho*(logM(rho)-logM(sigma))).tr()
    if abs(val.imag)>1.e-6:
        print("rho or sigma not positive")
        print(rho.eigenstates())
        print(sigma.eigenstates())
    return val.real



In [ ]:
def bures(rho, sigma):
    val = abs((sqrtM(rho)*sqrtM(sigma)).tr())
    val = max(min(val,1.),-1.)
    return np.arccos(val)/np.pi
        
def maxent_rho(rho, basis):   
    def test(x, rho, basis):
        k = sum([-u*b for u,b in zip(x, basis)])        
        sigma = (.5*(k+k.dag())).expm()
        sigma = sigma/sigma.tr()
        return rel_entropy(rho, sigma)    
    res = opt.minimize(test,np.zeros(len(basis)),args=(rho,basis))
    k = sum([-u*b for u,b in zip(res.x, basis)])        
    sigma = (.5*(k+k.dag())).expm()
    sigma = sigma/sigma.tr()
    return sigma
    
    
def error_maxent_state(rho, basis, distance=bures):
    try:
        sigma = maxent_rho(rho, basis)
        return distance(rho,sigma)
    except:
        print("fail")
        return None
    
    
def error_proj_state(rho, rho0, basis, distance=bures):
    try:
        basis = base_orth(basis, rho0)
        sigma = proj_op(logM(rho), basis, rho0).expm()
        return distance(rho, sigma)
    except:
        print("fail")
        return None

In [ ]:
id2 = qutip.qeye(2)
sx = .5*qutip.sigmax()
sy = .5*qutip.sigmay()
sz = .5*qutip.sigmaz()

sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    operator_list = []
    for m in range(N):
        operator_list.append(id2)
    operator_list[n] = sx
    sx_list.append(qutip.tensor(operator_list))
        
    operator_list[n] = sy
    sy_list.append(qutip.tensor(operator_list))
        
    operator_list[n] = sz
    sz_list.append(qutip.tensor(operator_list))

def Heisenberg_hamiltonian (N, Jx, Jy, Jz, h):
  H = 0
  for n in range(N):
    H += -0.5*h[n]*sz_list[n]
        
  for n in range(N-1):
    H += -0.5 * Jx[n] * sx_list[n] * sx_list[n+1]
    H += -0.5 * Jy[n] * sy_list[n] * sy_list[n+1]
    H += -0.5 * Jz[n] * sz_list[n] * sz_list[n+1]
  return H

N = 5            # número de spines

h  = 1.0 * 2 * np.pi * np.ones(N) 
Jx = 0.15 * 2 * np.pi * np.ones(N)
Jy = 0.25 * 2 * np.pi * np.ones(N)
Jz = 0.1 * 2 * np.pi * np.ones(N)

NameError: ignored

In [ ]:
def projected_evolution():